<a href="https://colab.research.google.com/github/ratneshpuri/Healthcare_Chatbot/blob/master/Healthcare_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy as np
import re

In [ ]:
nltk.download('punkt') # downloading model to tokenize message
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
nltk.download('stopwords') # downloading stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('wordnet') # downloading all lemmas of english language
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_corpus(data):
  corpus=[]
  for i in data:
    review=re.sub('[^a-zA-Z]',' ',i)
    review=review.lower()
    review=review.split()
    review=[lm.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)
  return corpus

In [ ]:
import json
with open('/content/health.json') as file:
    intents = json.load(file)

In [ ]:
corpus = []
tags = []
for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

cleaned_corpus=clean_corpus(corpus)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [ ]:
print(corpus)
len(tags)
print(len(corpus))
print(cleaned_corpus)

['Hi', 'How are you', 'Is there anyone?', ' Hello', 'Good Day', "What's up?", 'Bye', 'See you later!', 'Goodbye!', 'Thank you', 'Thanks', 'Thank you', "That's helpful", "Thank's a lot!", 'What do you do?', 'Who are you?', 'What are you here for?', 'What can you help me with?', 'Is anyone there?', 'Are you a real person?', 'Tell me about yourself!', 'I think I have anxiety', 'What is anxiety?', 'Tell me about anxiety', 'Can I have some information about anxiety?', 'How do I support a loved one with anxiety?', 'How do I fix my anxiety?', 'What other common signs and symptoms may occur with anxiety?', 'Symptoms of anxiety', 'signs of anxiety', 'What can I do to manage anxiety?', 'How to deal anxiety', 'How to overcome anxiety?', 'When should I call my doctor regarding anxity?', 'When should I take medical help to overcome anxiety', 'I think I have depression', 'What is depression?', 'Tell me about depression', 'Can I have some information about depression?', 'How do I support a loved one 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential([

Dense(128, input_shape=(X.shape[1],), activation='relu'), Dropout(0.2), Dense(64, activation='relu'), Dropout(0.2), 
Dense(y.shape[1], activation='softmax')])
model.compile(loss='categorical_crossentropy', optimizer='adam',
metrics=['accuracy'])
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
84/84 [==============================] - 0s 1ms/step - loss: 3.1251 - accuracy: 0.0833
Epoch 2/20
84/84 [==============================] - 0s 1ms/step - loss: 3.0276 - accuracy: 0.2262
Epoch 3/20
84/84 [==============================] - 0s 1ms/step - loss: 2.8714 - accuracy: 0.2143
Epoch 4/20
84/84 [==============================] - 0s 2ms/step - loss: 2.6390 - accuracy: 0.3214
Epoch 5/20
84/84 [==============================] - 0s 2ms/step - loss: 2.3536 - accuracy: 0.4405
Epoch 6/20
84/84 [==============================] - 0s 1ms/step - loss: 2.0320 - accuracy: 0.5000
Epoch 7/20
84/84 [==============================] - 0s 1ms/step - loss: 1.6087 - accuracy: 0.6429
Epoch 8/20
84/84 [==============================] - 0s 1ms/step - loss: 1.3443 - accuracy: 0.6667
Epoch 9/20
84/84 [==============================] - 0s 2ms/step - loss: 1.0678 - accuracy: 0.7738
Epoch 10/20
84/84 [==============================] - 0s 2ms/step - loss: 0.8353 - accuracy: 0.8214
Epoch 11/20
84/84 [

In [ ]:
model.save("health_version_0.0.1.h5")

In [ ]:
X.shape

(84, 60)

In [ ]:
y.shape

(84, 23)

In [ ]:
INTENT_NOT_FOUND_THRESHOLD = 0.40
def predict_intent_tag(message):
    message = clean_corpus([message])
    X_test = vectorizer.transform(message)
    y = model.predict(X_test.toarray())
    # if probability of all intent is low, classify it as noanswer
    if y.max() < INTENT_NOT_FOUND_THRESHOLD:
        return 'noanswer'
    prediction = np.zeros_like(y[0])
    prediction[y.argmax()] = 1
    tag = encoder.inverse_transform([prediction])[0][0]
    return tag
print(predict_intent_tag('Hi'))
print(predict_intent_tag('See you later'))
print(predict_intent_tag('Tell me about yourself'))

greetings
goodbye
about


In [ ]:
import random
import time
def get_intent(tag):
    # to return complete intent from intent tag
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return intent

In [ ]:
while True:
    # get message from user
    message = input('You: ')
    # predict intent tag using trained neural network
    tag = predict_intent_tag(message)
    # get complete intent from intent tag
    intent = get_intent(tag)
    # generate random response from intent
    response = random.choice(intent['responses'])
    print('Bot: ', response)
    # check if there's a need to perform some action
    if 'action' in intent.keys():
        action_code = intent['action']
        # perform action
        data = perform_action(action_code, intent)
        # get follow up intent after performing action
        followup_intent = get_intent(data['intent-tag'])
        # generate random response from follow up intent
        response = random.choice(followup_intent['responses'])
        # print randomly selected response
        if len(data.keys()) > 1:
            print('Bot: ', response.format(**data))
        else:
            print('Bot: ', response)
    # break loop if intent was goodbye
    if tag == 'goodbye':
        break

You: Who are your creator
Bot:  Hey! I love that question! My name is Doc, and I want to be your health friend and support you! My fellow human friends have trained me to be a compassionate listener and support buddy when things are going well, and particularily when things aren't going so well.
You: how to cure anxiety
Bot:  I see that you want to learn more about anxiety, and how to support yourself or your loved ones. At the higest level, anxiety is your body's natural response to stress. It's a feeling of fear or apprehension about what's to come. The first day of school, going to a job interview, or giving a speech may cause most people to feel fearful and nervous. We all get anxious sometimes, but anxiety becomes a problem when it starts affecting your daily life drastically. The best thing that you can do to help reduce anxiety is to take a few deep breaths and talk to someone about what you're feeling. Some natural remedies that you can try include: getting enough sleep meditat